# Test my own cityscpes code

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
from datetime import datetime
import os.path
import sys
import time

import numpy as np
from six.moves import xrange
import tensorflow as tf
import threading

from config import *
from dataset import pascal_voc, kitti, cityscapes
from utils.util import sparse_to_dense, bgr_to_rgb, bbox_transform
from nets import *

In [2]:
mc = cityscapes_squeezeDet_config()
print(mc)

dataset: cityscapes
dataset.upper: CITYSCAPES
{'NMS_THRESH': 0.4, 'LOSS_COEF_CONF_NEG': 100.0, 'KEEP_PROB': 0.5, 'IMAGE_WIDTH': 320, 'NUM_THREAD': 4, 'LOSS_COEF_CONF_POS': 75.0, 'GRID_POOL_HEIGHT': 7, 'PLOT_PROB_THRESH': 0.4, 'TOP_N_DETECTION': 64, 'LOSS_COEF_CLASS': 1.0, 'BGR_MEANS': array([[[103.939, 116.779, 123.68 ]]]), 'PROB_THRESH': 0.005, 'BATCH_SIZE': 20, 'DATASET': 'CITYSCAPES', 'DEBUG_MODE': False, 'MAX_GRAD_NORM': 1.0, 'EXP_THRESH': 1.0, 'DRIFT_X': 38, 'LEARNING_RATE': 0.01, 'CLASS_NAMES': ['car', 'person', 'bicycle'], 'LEAKY_COEF': 0.1, 'EXCLUDE_HARD_EXAMPLES': False, 'QUEUE_CAPACITY': 100, 'DATA_AUGMENTATION': True, 'GRID_POOL_WIDTH': 7, 'PRETRAINED_MODEL_PATH': '', 'CLASSES': 3, 'IS_TRAINING': False, 'ANCHOR_BOX': array([[  4.05063291,   6.4       ,  36.        ,  37.        ],
       [  4.05063291,   6.4       , 366.        , 174.        ],
       [  4.05063291,   6.4       , 115.        ,  59.        ],
       ...,
       [315.94936709, 153.6       , 224.        , 108. 

In [3]:
imdb = cityscapes('train', '/home/zhengyuanbo/project/SQDT_ROOT/data/cityscapes/', mc)

image_set: train
data_root_path: /home/zhengyuanbo/project/SQDT_ROOT/data/cityscapes/
image_path: /home/zhengyuanbo/project/SQDT_ROOT/data/cityscapes/label
classes: ['car', 'person', 'bicycle']
class_to_idx: {'car': 0, 'bicycle': 2, 'person': 1}
image_idx: ['aachen/aachen_000000_000019', 'aachen/aachen_000002_000019', 'aachen/aachen_000004_000019', 'aachen/aachen_000005_000019', 'aachen/aachen_000001_000019', 'aachen/aachen_000009_000019', 'aachen/aachen_000003_000019', 'aachen/aachen_000006_000019', 'aachen/aachen_000007_000019', 'aachen/aachen_000008_000019', 'bbchen/aachen_000000_000019', 'bbchen/aachen_000002_000019', 'bbchen/aachen_000004_000019', 'bbchen/aachen_000005_000019', 'bbchen/aachen_000001_000019', 'bbchen/aachen_000009_000019', 'bbchen/aachen_000003_000019', 'bbchen/aachen_000006_000019', 'bbchen/aachen_000007_000019', 'bbchen/aachen_000008_000019']
rois: {'aachen/aachen_000000_000019': [[708.5, 476.5, 199.0, 113.0, 0], [225.0, 466.0, 160.0, 74.0, 0], [225.0, 466.0, 160